<a href="https://colab.research.google.com/github/egonik-unlp/tp_curso_ml/blob/main/src/notebooks/autoencoder_callback_newdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import requests
import json
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import normalize, MinMaxScaler
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
codigo_numerico=requests.get("https://raw.githubusercontent.com/egonik-unlp/tp_curso_ml/main/src/notebooks/codigo_numerico.json").json()

In [ ]:
codigo_numerico=requests.get('https://raw.githubusercontent.com/egonik-unlp/tp_curso_ml/d61d2cd55fe79750c892a739129de1e02ead2d29/src/pickle/json_files/codigo_blosum62.json').json()

In [ ]:
%cd /content/drive/MyDrive/protein_classifier_with_locations/dataset_proteoma_full/
proteome=np.load("proteome_short.npy")

/content/drive/MyDrive/protein_classifier_with_locations/dataset_proteoma_full


In [ ]:
# scaler=MinMaxScaler()
X_train_val, X_test=train_test_split(proteome)
X_train, X_valid=train_test_split(X_train_val)

# Por el momento voy a tratar de escalar manualmente, para evitar problemas de redondeo en la
# conversión inversa

# X_train_n=scaler.fit_transform(X_train)
# X_valid_n=scaler.transform(X_valid)
# X_test_n=scaler.transform(X_test)


X_train_n=X_train/25 # el valor maximo del 'codigo numerico'
X_valid_n=X_valid/25
X_test_n=X_test/25

In [ ]:
# Reshapeo para que funcione en el AE
X_train_bis=X_train_n.reshape(-1,2000,1)
X_valid_bis=X_valid_n.reshape(-1,2000,1)
X_test_bis=X_test_n.reshape(-1,2000,1)
del X_train_n
del X_valid_n
del X_test_n

In [ ]:
X_train_bis.shape

(56250, 2000, 1)

In [ ]:
tf.keras.backend.clear_session()

conv_encoder = keras.models.Sequential([
keras.layers.Reshape([2000, 1], input_shape=[2000,1]),
keras.layers.Conv1D(16, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2),
keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2),
keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2)
])


conv_decoder = keras.models.Sequential([
keras.layers.Conv1DTranspose(32, kernel_size=3, strides=2, padding="same",
activation="selu",
input_shape=[250, 64]),
keras.layers.Conv1DTranspose(16, kernel_size=3, strides=2, padding="same",
activation="selu"),
keras.layers.Conv1DTranspose(1, kernel_size=3, strides=2, padding="same",
activation="sigmoid"),

])


conv_ae = keras.models.Sequential([conv_encoder, conv_decoder])
conv_ae.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 250, 64)           7840      
_________________________________________________________________
sequential_1 (Sequential)    (None, 2000, 1)           7777      
Total params: 15,617
Trainable params: 15,617
Non-trainable params: 0
_________________________________________________________________


In [ ]:
nums_codes={v:k for k,v in codigo_numerico.items()}

In [ ]:
def unpad(query, target):
  try:
    idx=np.where(query==0)[0][-1] + 1
  except IndexError:
    idx=0
  return query[idx:], target[idx:]
  
class ProteinEvaluator(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}, protein_code=nums_codes):
    if epoch%5 == 0 and epoch!=0:
      predictions=self.model.predict(X_valid_bis)
      pred_seqs=[]
      for n,(query, target) in enumerate(zip(X_valid_bis.reshape(-1,2000), predictions.reshape(-1,2000))):
        qry,tgt=unpad(query,target)
        del qry
        pred_seqs.append(tgt)
      compressed_proteins=[''.join(np.vectorize(lambda x:nums_codes.get(round(x*25),'X'))(seq)) for seq in pred_seqs]
      with open('/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/sequences_checkpoints/pred_seqs_epoch_{}.json'.format(epoch), 'w') as file:
        json.dump(compressed_proteins, file)
pe=ProteinEvaluator()

In [ ]:
0%5

0

In [ ]:
class CustomSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch %5 == 0:  # or save after some epoch, each k-th epoch etc.
            self.model.save('/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/autoencoder_prot_{}.h5'.format(epoch))
cs=CustomSaver()

In [ ]:
conv_ae.compile(loss="binary_crossentropy",
optimizer=keras.optimizers.SGD(learning_rate=1.5))
# checkpoint = keras.callbacks.ModelCheckpoint(f'/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/autoencoder_prot_{epoch:08d}.h5', period=5)
history = conv_ae.fit(X_train_bis, X_train_bis, epochs=150,
  validation_data=[X_valid_bis, X_valid_bis], callbacks=[pe])

Epoch 1/150
1758/1758 [==============================] - 50s 11ms/step - loss: 0.0986 - val_loss: 0.0000e+00
Epoch 2/150
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0855 - val_loss: 0.0000e+00
Epoch 3/150
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0836 - val_loss: 0.0000e+00
Epoch 4/150
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0827 - val_loss: 0.0000e+00
Epoch 5/150
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0822 - val_loss: 0.0000e+00
Epoch 6/150
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0818 - val_loss: 0.0000e+00
Epoch 7/150
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0816 - val_loss: 0.0000e+00
Epoch 8/150
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0814 - val_loss: 0.0000e+00
Epoch 9/150
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0812 - val_loss: 0.0000e+00
Epoch 10/150
1758/1

In [ ]:
np.save('/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/sequences_checkpoints/arrays/X_valid_bis', X_valid_bis)

In [ ]:
!cd /content/drive/MyDrive/protein_classifier_with_locations/autoencoder/sequences_checkpoints/

/bin/bash: line 0: cd: /content/drive/MyDrive/protein_classifier_with_locations/autoencoder/sequences_checkpoints/: No such file or directory


In [ ]:
def unpad(query, target):
  try:
    idx=np.where(query==0)[0][-1] + 1
  except IndexError:
    idx=0
  return query[idx:], target[idx:]

In [ ]:
rpts=4
pred_seqs=[]
original_seqs=[]

for i in range(rpts):
  history = conv_ae.fit(X_train_bis, X_train_bis, epochs=10,
  validation_data=[X_valid_bis, X_valid_bis])
  scores=np.zeros(X_test_bis.shape[0])
  predictions=conv_ae.predict(X_test_bis)
  for n,(query, target) in enumerate(zip(X_test_bis.reshape(-1,2000), predictions.reshape(-1,2000))):
    qry,tgt=unpad(query,target)
    original_seqs.append(qry)
    del qry
    pred_seqs.append(tgt)
  del predictions


Epoch 1/10
1758/1758 [==============================] - 50s 11ms/step - loss: 0.0997 - val_loss: 0.0000e+00
Epoch 2/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0859 - val_loss: 0.0000e+00
Epoch 3/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0839 - val_loss: 0.0000e+00
Epoch 4/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0829 - val_loss: 0.0000e+00
Epoch 5/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0823 - val_loss: 0.0000e+00
Epoch 6/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0819 - val_loss: 0.0000e+00
Epoch 7/10
1758/1758 [==============================] - 20s 11ms/step - loss: 0.0816 - val_loss: 0.0000e+00
Epoch 8/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0814 - val_loss: 0.0000e+00
Epoch 9/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0812 - val_loss: 0.0000e+00
Epoch 10/10
1758/1758 [=====

In [ ]:
original_seqs=[seq for seq in X_test_bis.reshape(-1,2000)]

In [ ]:
nums_codes={v:k for k,v in codigo_numerico.items()} 
# compressed_proteins=[''.join(np.vectorize(lambda x:nums_codes.get(round(x*25),'X'))(seq)) for seq in pred_seqs]
original_proteins=[''.join(np.vectorize(lambda x:nums_codes.get(round(x*25),'X'))(seq)) for seq in original_seqs]

In [ ]:
len(original_proteins)

25000

In [ ]:
# np.save('/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/predictions',predictions)
np.save('/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/compressed_proteins',compressed_proteins)
np.save('/content/drive/MyDrive/protein_classifier_with_locations/autoencoder/original_proteins', original_proteins)